# Filter out binge entries from mEMA questionnaire

First drop all non-binge instances from app data

In [ ]:

import pandas as pd

path_in = '../resources/data_ignored/mEMA_Qualtrics_subject_part/report.merged-sanne-binge_1705.xlsx'

df = pd.read_excel(path_in)
col_subject_id = 'subject_id_x'

print('unique', col_subject_id, len(df[col_subject_id].unique()))
print('total instances', len(df.index))
df_copy = df
df = df.drop(df.index[df['objective_binge'] != True])
print('binge instances', len(df.index))

For binge instances determine unique subject ids

In [ ]:
print('unique binge', col_subject_id, len(df[col_subject_id].unique()))
objective_binge_subject_ids = df[col_subject_id].unique()
print(objective_binge_subject_ids)

Map subject ids to participant ids using a lookup excel document

In [ ]:
path_in_mapping = '../resources/data_ignored/mEMA_Qualtrics_subject_part/Mapping_participant_subjectID.xlsx'
path_in_questionnaire = '../resources/data_ignored/mEMA_Qualtrics_subject_part/Qualtrics_Merged_OBE_without_POST.xlsx'
path_in_questionnaire_out = '../resources/data_ignored/mEMA_Qualtrics_subject_part/Qualtrics_Merged_OBE_without_POST_merge.xlsx'

col_participant = 'participant' # please keep column names equal in mapping

df_mapping = pd.read_excel(path_in_mapping)
df_q = pd.read_excel(path_in_questionnaire)

def get_mapped_participants():
    objective_binge_participants = []
    for subject_id in objective_binge_subject_ids:
        value = df_mapping.loc[df_mapping[col_subject_id] == subject_id, col_participant].iloc[0]
        objective_binge_participants.append(value)
        print(subject_id, value)
    return objective_binge_participants

objective_binge_participants = get_mapped_participants()



Drop all paricipants without binges and write to new excel file

In [ ]:
df_q_copy = df_q
print(len(df_q.index))
print('unique', col_participant, len(df_q[col_participant].unique()))
df_q = df_q[df_q[col_participant].isin(objective_binge_participants)]
print(len(df_q.index))
print('unique binge', col_participant, len(df_q[col_participant].unique()))

df_q.to_excel(path_in_questionnaire_out)

Add columns to eating moments

In [ ]:
def get_participant_value(subject_id, column_name):
    mapped_participant = df_mapping.loc[df_mapping[col_subject_id] == subject_id, col_participant]
    if len(mapped_participant.value_counts()) == 0:
        return 'NO_MAPPING'
    
    mapped_participant_value = mapped_participant.iloc[0]
    val = df_q_copy.loc[df_q_copy[col_participant] == mapped_participant_value, column_name]
    if len(val.value_counts()) == 0:
        return 'NO_VALUE'
    return val.iloc[0]

columns_to_add = ['ClassificationEDEQ']
for column_name in columns_to_add:
    print(column_name)
    df_copy[column_name] = df_copy.apply(lambda x: get_participant_value(x[col_subject_id], column_name), axis=1)
   
print('write')
df_copy.to_excel('../resources/data_ignored/mEMA_Qualtrics_subject_part/report.merged-sanne-binge_1705-out.xlsx')  
print('done!')